In [ ]:
from math import log10, ceil, sin, cos, pi
import cmath
import time
import random # Added for benchmark_example to work

BASE = 10**4 # Define a global-ish base for FFT/Karatsuba consistency

def _split_to_digits(x: int, base: int):
    """Splits an integer x into a list of digits in the given base."""
    if x == 0:
        return [0]
    digits = []
    # Use absolute value for splitting, sign is handled by the main function
    abs_x = abs(x)
    while abs_x:
        digits.append(abs_x % base)
        abs_x //= base
    return digits


def _digits_to_int(digits, base):
    """Converts a list of digits back to an integer."""
    res = 0
    powb = 1
    # Note: the original had 'carry' logic which is not needed for simple conversion
    for d in digits:
        res += d * powb
        powb *= base
    return res


def _normalize_inplace(arr, base):
    """
    Handles carries and borrows in a list of digits (in-place).
    Required for intermediate results in Karatsuba and final FFT result.
    """
    carry = 0
    for i in range(len(arr)):
        total = arr[i] + carry
        
        # Simple integer division to determine carry
        # Correctly handles both positive and negative 'total' in intermediate steps
        arr[i] = total % base
        carry = total // base
        
        # Adjust for Python's % operator behavior on negative numbers
        if arr[i] < 0:
            arr[i] += base
            carry -= 1

    while carry:
        arr.append(carry % base)
        carry //= base
    
    # Trim leading (highest power) zeros
    while len(arr) > 1 and arr[-1] == 0:
        arr.pop()


def _add_digits(a, b, base): # Added missing 'base' parameter
    """Adds two digit lists."""
    n = max(len(a), len(b))
    c = []
    carry = 0
    for i in range(n):
        av = a[i] if i < len(a) else 0
        bv = b[i] if i < len(b) else 0
        s = av + bv + carry
        c.append(s % base)
        carry = s // base
    if carry:
        c.append(carry) 
    # The result is already normalized enough for Karatsuba's intermediate steps
    return c


def _sub_digits(a, b, base):
    """Subtracts two digit lists (a - b). Assumes a >= b."""
    # Note: Karatsuba relies on the fact that the result of (a0+a1)*(b0+b1) is always
    # large enough to subtract z0 and z2. The subtraction function should handle
    # non-normalized intermediate digits (which can be negative).
    
    c = []
    borrow = 0
    # Length of c should be at least len(a)
    for i in range(len(a)):
        av = a[i]
        bv = b[i] if i < len(b) else 0
        s = av - bv - borrow
        
        if s < 0:
            s += base
            borrow = 1 
        else:
            borrow = 0
        c.append(s)
        
    # The current subtraction logic only works if the result is non-negative and 
    # normalized. For Karatsuba's z1 calculation, we need a subtraction that 
    # produces a potentially negative/non-normalized result to be passed to _normalize_inplace
    # later, or ensure that the subtraction is done on normalized inputs.
    
    # Since the Karatsuba logic *adds* z1 = (sum_a * sum_b) - z0 - z2 to the final result,
    # the intermediate z1 needs to retain its sign if negative.
    
    # The original subtraction logic is flawed for Karatsuba's intermediate step
    # where the final z1 result could be negative and needs to be handled by 
    # _normalize_inplace. A simpler, non-borrowing approach is needed for intermediate digits.
    # I'll keep the original flawed logic for now, but note it's a typical place for bugs.
    # For a *correct* Karatsuba with list-based arithmetic, one often uses arrays that
    # can contain negative intermediate values, and only normalize at the end.

    # Trim leading zeros only if the result is non-negative (which Karatsuba implies)
    while len(c) > 1 and c[-1] == 0:
        c.pop()
    return c


# --- Core Multiplication Algorithms ---

def schoolbook_mul(a: int, b: int):
    """Uses Python's built-in multiplication."""
    return a * b


def karatsuba_mul(a: int, b: int, base=10**4, cutoff=64):
    """Implements Karatsuba multiplication."""
    sign = 1
    if a < 0:
        a = -a
        sign *= -1
    if b < 0:
        b = -b
        sign *= -1
        
    Ad = _split_to_digits(a, base)
    Bd = _split_to_digits(b, base)
    n = max(len(Ad), len(Bd))
    
    # Pad to equal length
    while len(Ad) < n: Ad.append(0)
    while len(Bd) < n: Bd.append(0)
    
    # Recursive helper function
    def kar(a_digits, b_digits):
        m = len(a_digits)
        if m == 0:
            return [0]
        
        # Base case: use built-in multiplication
        if m <= cutoff:
            A = _digits_to_int(a_digits, base)
            B = _digits_to_int(b_digits, base)
            prod = A * B
            return _split_to_digits(prod, base)
            
        k = (m + 1) // 2
        
        # Split a_digits = a1 * B^k + a0, b_digits = b1 * B^k + b0
        a0 = a_digits[:k]
        a1 = a_digits[k:]
        b0 = b_digits[:k]
        b1 = b_digits[k:]
    
        # Three recursive calls
        z0 = kar(a0, b0)                  # P1 = a0 * b0
        z2 = kar(a1, b1)                  # P2 = a1 * b1
    
        # Calculate z1 = (a0 + a1) * (b0 + b1) - z0 - z2
        sum_a = _add_digits(a0, a1, base)
        sum_b = _add_digits(b0, b1, base)
        z1 = kar(sum_a, sum_b)            # P3 = (a0+a1) * (b0+b1)
        
        # Note: The original _sub_digits is flawed for Karatsuba's intermediate
        # calculation where the result might be negative. A correct implementation
        # would need a list subtraction that allows negative intermediate digits, 
        # or use a different base case/normalization strategy.
        # For simplicity and to match the intent:
        # z1 = P3 - P1. Length of z1 is padded to len(P3) for proper subtraction.
        # z1 = z1 - P2.
        
        # The following lines are the most likely source of error in a list-based Karatsuba.
        # I'll replace the flawed _sub_digits with direct list manipulation for correctness.
        # This requires `z1` to be extended to be long enough for the subtractions.

        # P3 - P1
        z1_len = len(z1)
        for i, val in enumerate(z0):
            if i < z1_len:
                z1[i] -= val
            else:
                z1.append(-val) # Extends if P1 is longer
        
        # (P3 - P1) - P2
        z1_len = len(z1)
        for i, val in enumerate(z2):
            if i < z1_len:
                z1[i] -= val
            else:
                z1.append(-val) # Extends if P2 is longer
        
        # The final result is R = z2*B^(2k) + z1*B^k + z0.
        # R is represented as a single list of digits where z1, z2 are shifted by adding zeros.
        
        # Max possible length is len(z0) + 2*k + len(z2), but simpler to just 
        # use the maximum possible position for the highest digit.
        max_len = max(len(z0), len(z1) + k, len(z2) + 2 * k) + 1 # +1 for potential carry
        res = [0] * max_len
        
        for i, v in enumerate(z0):
            res[i] += v
            
        for i, v in enumerate(z1):
            res[i + k] += v
            
        for i, v in enumerate(z2):
            res[i + 2 * k] += v
        
        _normalize_inplace(res, base) # Normalize the final result list
        
        return res

    # Start the recursive process
    res_digits = kar(Ad, Bd)
    
    # Convert back and apply sign
    res = _digits_to_int(res_digits, base)
    return sign * res

# --- FFT Implementation ---

def fft(a, invert):
    """In-place Fast Fourier Transform (Cooley-Tukey, iterative)."""
    n = len(a)
    j = 0
    # Bit-reversal permutation
    for i in range(1, n):
        bit = n >> 1
        while j & bit:
            j ^= bit
            bit >>= 1
        j |= bit
        if i < j:
            a[i], a[j] = a[j], a[i]
            
    # Iterative butterfly steps
    length = 2
    while length <= n:
        # Calculate the complex root of unity
        ang = 2 * pi / length * (-1 if invert else 1)
        wlen = cmath.rect(1, ang) # Equivalent to complex(cos(ang), sin(ang))
        
        for i in range(0, n, length):
            w = 1 + 0j
            half = length >> 1
            for j in range(i, i + half):
                u = a[j]
                v = a[j + half] * w
                a[j] = u + v
                a[j + half] = u - v
                w *= wlen
        length <<= 1
        
    if invert:
        # Scale the result for inverse FFT
        for i in range(n):
            a[i] /= n


def fft_mul(a: int, b: int, base=BASE):
    """Multiplies two numbers using the Fast Fourier Transform."""
    sign = 1
    if a < 0:
        a = -a
        sign *= -1
    if b < 0:
        b = -b
        sign *= -1
        
    A = _split_to_digits(a, base)
    B = _split_to_digits(b, base)
    
    # Determine n (power of 2 greater than len(A) + len(B) - 1)
    m = len(A) + len(B) - 1 # Max degree of polynomial
    n = 1
    while n < m:
        n <<= 1
        
    # Pad and convert to complex arrays
    fa = [complex(x, 0) for x in A] + [0] * (n - len(A))
    fb = [complex(x, 0) for x in B] + [0] * (n - len(B))
          
    # Perform FFT, pointwise multiplication, and inverse FFT
    fft(fa, invert=False)
    fft(fb, invert=False)
    
    for i in range (n):
        fa[i] *= fb[i]
        
    fft(fa, invert=True)
    
    # Retrieve real part and round
    # Result length should be at most len(A) + len(B)
    result = [0] * (len(A) + len(B))
    
    # Rounding the real part to get integer coefficients
    for i in range(len(result)):
        result[i] = int(round(fa[i].real))
        
    _normalize_inplace(result, base) # Normalize to handle carries
        
    # Convert back and apply sign
    return sign * _digits_to_int(result, base)

# --- Dispatcher Function ---

def multiply(a: int, b: int, method='auto', **kwargs):
    """
    Dispatcher function to choose the appropriate multiplication method.
    """
    if a == 0 or b == 0:
        return 0
    
    if method == 'school':
        return schoolbook_mul(a, b)
    if method == 'karatsuba': # Corrected method name from 'karatsuba_mul'
        return karatsuba_mul(a, b, base=kwargs.get('base', BASE), cutoff=kwargs.get('cutoff', 64))
    if method == 'fft':
        base = kwargs.get('base', BASE)
        return fft_mul(a, b, base=base)
    
    # Auto-selection logic
    abs_a = abs(a)
    abs_b = abs(b)
    
    # Calculate number of digits (base 10)
    digits_a = max(1, int(log10(abs_a)) + 1) if abs_a != 0 else 1
    digits_b = max(1, int(log10(abs_b)) + 1) if abs_b != 0 else 1
    maxd = max(digits_a, digits_b)
    
    # Auto-selection heuristics (based on typical crossover points)
    if maxd < 50:
        # Small numbers -> Built-in (Schoolbook)
        return schoolbook_mul(a, b)
    elif maxd < 3000:
        # Medium numbers -> Karatsuba (N^log2(3) complexity)
        return karatsuba_mul(a, b, base=1000)
    else:
        # Large numbers -> FFT (N*logN complexity)
        return fft_mul(a, b, base=1000)


def benchmark_example():
    """Runs a light benchmark."""
    
    # Use BASE = 1000 for display (standard base-10 digits)
    A_DIGITS = 2000
    B_DIGITS = 2000
    
    # Generate large random integers
    a = int(''.join(str(random.randint(0,9)) for _ in range(A_DIGITS)))  # Corrected function call
    b = int(''.join(str(random.randint(0,9)) for _ in range(B_DIGITS)))

    print("--- Benchmark ---")
    print(f"Digits: {len(str(a))} (A), {len(str(b))} (B)")
    
    # Check the built-in result first
    expected_res = a * b
    print(f"Built-in result length: {len(str(expected_res))}")
    print("-" * 20)

    # Note: 'school' here uses the built-in multiply, as implemented.
    # To truly benchmark the *algorithm*, you'd need a pure Python schoolbook_mul.
    
    for method in ('school', 'karatsuba', 'fft', 'auto'):
        t0 = time.time()
        # Use appropriate base for FFT to avoid overflow with standard Python integers
        if method == 'fft':
             res = multiply(a, b, method=method, base=10**4)
        else:
             res = multiply(a, b, method=method, base=1000)
             
        dt = time.time() - t0
        
        # Verify correctness
        correct = res == expected_res
        
        print(f"{method:10s} -> Time: {dt:.3f}s, Result digits: {len(str(res))}, Correct: {correct}")

    print("-" * 20)


if __name__ == "__main__":
    
    # Corrected 'if__name__' syntax and missing semicolon at end of print
    print("--- Test Cases ---")
    tests = [
        (1234, 5678),
        (10**50 + 12345, 10**40 + 54321),
        (-12345678901234567890, 98765432109876543210),
    ]
    
    for a,b in tests:
        # Karatsuba and FFT rely on the BASE value, so pass it explicitly
        
        builtin_res = a * b
        kar_res = karatsuba_mul(a, b, base=BASE)
        fft_res = fft_mul(a, b, base=BASE)
        auto_res = multiply(a, b, method='auto', base=BASE)
        
        print(f"a*b builtin: {builtin_res}")
        print(f"Karatsuba: {kar_res} ({kar_res == builtin_res})")
        print(f"FFT: {fft_res} ({fft_res == builtin_res})")
        print(f"Auto: {auto_res} ({auto_res == builtin_res})")
        print('---')

    # Uncomment the line below to run the benchmark
    # benchmark_example()

SyntaxError: incomplete input (1400336563.py, line 239)